<center><img src="https://raw.githubusercontent.com/mateuszk098/kaggle_notebooks/master/playground_series_s3e21/undraw_Programming_re_kg9v.png" width=600px></center>

# <p style="font-family: 'JetBrains Mono'; font-weight: bold; font-size: 125%; color: #4A4B52; text-align: center">Playground Series S3E21 - Dissolved O2 in River Water</p>


In [1]:
# %load ../general_settings.py
import glob
import os
import shutil
import subprocess
import warnings
from array import array
from collections import defaultdict, namedtuple
from copy import copy
from functools import partial
from itertools import chain, combinations, product
from pathlib import Path
from time import strftime

ON_KAGGLE = os.getenv("KAGGLE_KERNEL_RUN_TYPE") is not None
if ON_KAGGLE:
    warnings.filterwarnings("ignore")

import joblib
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import scipy.stats as stats
import seaborn as sns
import shap

# import swifter
from colorama import Fore, Style
from IPython.core.display import HTML, display_html
from plotly.subplots import make_subplots
from scipy.cluster.hierarchy import linkage
from scipy.spatial.distance import squareform

# Colorama settings.
CLR = (Style.BRIGHT + Fore.BLACK) if ON_KAGGLE else (Style.BRIGHT + Fore.WHITE)
RED = Style.BRIGHT + Fore.RED
BLUE = Style.BRIGHT + Fore.BLUE
CYAN = Style.BRIGHT + Fore.CYAN
YELLOW = Style.BRIGHT + Fore.YELLOW
RESET = Style.RESET_ALL

FONT_COLOR = "#545454"
BACKGROUND_COLOR = "#FFFCFA"

CELL_HOVER = {  # for row hover use <tr> instead of <td>
    "selector": "td:hover",
    "props": "background-color: #FFFCFA",
}
TEXT_HIGHLIGHT = {
    "selector": "td",
    "props": "color: #4A4B52; font-weight: bold",
}
INDEX_NAMES = {
    "selector": ".index_name",
    "props": "font-weight: normal; background-color: #FFFCFA; color: #4A4B52;",
}
HEADERS = {
    "selector": "th:not(.index_name)",
    "props": "font-weight: normal; background-color: #FFFCFA; color: #4A4B52;",
}
DF_STYLE = (INDEX_NAMES, HEADERS, TEXT_HIGHLIGHT)
DF_CMAP = sns.light_palette("#BAB8B8", as_cmap=True)

# Utility functions.
def download_from_kaggle(expr: list[str], directory: Path | None = None) -> None:
    if directory is None:
        directory = Path("data")
    if not isinstance(directory, Path):
        raise TypeError("The `directory` argument must be `Path` instance!")
    match expr:
        case ["kaggle", _, "download", *args] if args:
            directory.parent.mkdir(parents=True, exist_ok=True)
            filename = args[-1].split("/")[-1] + ".zip"
            if not (directory / filename).is_file():
                subprocess.run(expr)
                shutil.unpack_archive(filename, directory)
                shutil.move(filename, directory)
        case _:
            raise SyntaxError("Invalid expression!")


def interpolate_color(color1, color2, t):
    r1, g1, b1 = int(color1[1:3], 16), int(color1[3:5], 16), int(color1[5:7], 16)
    r2, g2, b2 = int(color2[1:3], 16), int(color2[3:5], 16), int(color2[5:7], 16)
    r = int(r1 + (r2 - r1) * t)
    g = int(g1 + (g2 - g1) * t)
    b = int(b1 + (b2 - b1) * t)
    return f"#{r:02X}{g:02X}{b:02X}"


def get_interpolated_colors(color1, color2, num_colors=2):
    """Return `num_colors` interpolated beetwen `color1` and `color2`.
    Arguments need to be HEX."""
    num_colors = num_colors + 2
    return [interpolate_color(color1, color2, i / (num_colors - 1)) for i in range(num_colors)]


# Html `code` block highlight. Must be included at the end of all imports!
HTML(
    """
<style>
code {
    background: rgba(42, 53, 125, 0.10) !important;
    border-radius: 4px !important;
}
a {
    color: rgba(123, 171, 237, 1.0) !important;
}
ol.numbered-list {
  counter-reset: item;
}
ol.numbered-list li {
  display: block;
}
ol.numbered-list li:before {
  content: counters(item, '.') '. ';
  counter-increment: item;
}
</style>
"""
)


<p style="
    font-size: 20px;
    font-family: 'JetBrains Mono';
    color: #4A4B52;
">
    <b>Competition Description</b> 📜
</p>

<p style="
    font-size: 16px;
    font-family: 'JetBrains Mono';
">
    The dataset for this competition is a synthetic dataset based off of the <a href="https://www.kaggle.com/datasets/vbmokin/dissolved-oxygen-prediction-in-river-water"><b>Dissolved oxygen prediction in river water dataset</b></a>. As we can read in the original dataset description: <i>This dataset has data of the 5 indicators of river water quality from 8 consecutive stations of the state water monitoring system. It's should predict the value in the eighth station by the first seven stations. The numbering of stations in the dataset is done from the target station upstream, ie closest to it - first, upstream - second, etc. Data are average monthly. The number of observations on stations is different (from 4 to about 20 years). Training and test data are chosen so that the percentage of non-NA values on stations with long and short series data is approximately the same.</i>
</p>
    
<p style="
    font-size: 20px;
    font-family: 'JetBrains Mono';
    color: #4A4B52;
">
    <b>Task</b> 💡
</p>

<p style="
    font-size: 16px;
    font-family: 'JetBrains Mono';
">
    This is a different type of competition. <b>Instead of submitting predictions, your task is to submit a dataset that will be used to train a random forest regressor model.</b> Your submission will be used as a training dataset to train the following model and make predictions against a hidden test dataset:<br><br>
    <code style="
        display: block;
        white-space: pre-wrap;
    ">
    from sklearn.ensemble import RandomForestRegressor<br>
    y_train = train.pop("target")  # train is your submission!
    rf = RandomForestRegressor(n_estimators=1000, max_depth=7, n_jobs=-1, random_state=42)
    rf.fit(train, y_train)
    y_hat = rf.predict(test)  # test set is hidden from you
    </code><br>
    The competition evaluation metric is <a href="https://en.wikipedia.org/wiki/Root-mean-square_deviation"><b>RMSE</b></a> (root mean squared error):
    \[\textrm{RMSE} = \sqrt{\frac{1}{N}\sum_{i=1}^N \left(y_i - \hat{y}_i\right)^2},\]
    where $y_i$ is the actual value and $\hat{y}_i$ is the predicted value, for the sample $i$.
</p>

<p style="
    font-size: 20px;
    font-family: 'JetBrains Mono';
    color: #4A4B52;
">
    <b>Table of Contents</b> 📔
</p>

<p style="
    font-size: 16px;
    font-family: 'JetBrains Mono';
">
    The table of contents provides pleasurable navigation through the whole notebook. You can easily navigate through sections and return to TOC. If you want quickly find out something about the dataset, just read the first section, i.e. <b>Quick Overview</b>.
</p>

<blockquote class="anchor" id="top" style="
    margin-right: auto; 
    margin-left: auto;
    padding: 10px;
    background-color: #FF7F51;
    border-radius: 2px;
    border: 1px solid #FF7F51;
">
<ol class="numbered-list" style="
    font-size: 16px;
    font-family: 'JetBrains Mono';
    color: #F2F2F0;
    margin-top: 15px;
    margin-bottom: 15px;
">
    <li><a href="#quick_overview"><span style="color: #F2F2F0">Quick Overview</span></a>
    <ol class="numbered-list" class="numbered-list" style="
        font-size: 16px;
        font-family: 'JetBrains Mono';
        color: #F2F2F0;
    ">
        <li><a href="#data_reading_and_features_description"><span style="color: #F2F2F0">Data Reading &amp; Features Description</span></a></li>
        <li><a href="#basic_numerical_relations_summaries"><span style="color: #F2F2F0">Basic Numerical Relations &amp; Summaries</span></a></li>
        <li><a href="#distributions"><span style="color: #F2F2F0">Distributions</span></a></li>
        <li><a href="#quick_summary"><span style="color: #F2F2F0">Quick Summary</span></a></li>
    </ol>
    </li>
    <li><a href="#feature_importances"><span style="color: #F2F2F0">Feature Importances</span></a>
</ol>
</blockquote>


# <b> <span style="font-family: 'JetBrains Mono'; color: #4A4B52">1</span> <span style='color: #FF7F51'>|</span> <span style="font-family: 'JetBrains Mono'; color: #4A4B52">Quick Overview</span></b><a class="anchor" id="quick_overview"></a> [↑](#top)


<p style="
    font-size: 20px;
    font-family: 'JetBrains Mono';
    color: #4A4B52;
    border-bottom: 2px solid #FF7F51;
">
    <b>About Section</b> 💡
</p>

<p style="
    font-size: 16px;
    font-family: 'JetBrains Mono';
">
    In this section, I provide a quick overview of the dataset. More detailed analysis will be done in subsequent sections.
</p>

## <b> <span style="font-family: 'JetBrains Mono'; color: #4A4B52">1.1</span> <span style='color: #FF7F51'>|</span> <span style="font-family: 'JetBrains Mono'; color: #4A4B52">Data Reading &amp; Features Description</span></b><a class="anchor" id="data_reading_and_features_description"></a> [↑](#top)

<p style="
    font-size: 16px;
    font-family: 'JetBrains Mono';
">
    <b>Let's get started with loading the dataset and explaining what available features mean.</b>
</p>

In [21]:
competition = "playground-series-s3e21"
expr = f"kaggle competitions download -c {competition}".split()

if not ON_KAGGLE:
    download_from_kaggle(expr)
    train_path = "data/sample_submission.csv"
else:
    train_path = f"/kaggle/input/{competition}/sample_submission.csv"

train = pd.read_csv(train_path, index_col="id", engine="pyarrow").rename(columns=str.title)


<p style="
    font-size: 20px;
    font-family: 'JetBrains Mono';
    color: #4A4B52;
    border-bottom: 2px solid #FF7F51;
">
    <b>Insight</b> 💡
</p>

<p style="
    font-size: 16px;
    font-family: 'JetBrains Mono';
">
    I prefer to capitalize data frame columns because sometimes they can contain features named the same way as some methods. Since I like to refer to columns as an attribute (with a dot), it's better to provide capitalized names.
</p>

In [24]:
train.head().style.set_table_styles(DF_STYLE).format(precision=5)


,Target,O2_1,O2_2,O2_3,O2_4,O2_5,O2_6,O2_7,Nh4_1,Nh4_2,Nh4_3,Nh4_4,Nh4_5,Nh4_6,Nh4_7,No2_1,No2_2,No2_3,No2_4,No2_5,No2_6,No2_7,No3_1,No3_2,No3_3,No3_4,No3_5,No3_6,No3_7,Bod5_1,Bod5_2,Bod5_3,Bod5_4,Bod5_5,Bod5_6,Bod5_7
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,8.59000,7.50000,9.00000,9.54500,9.26500,8.11000,8.43000,7.15000,0.18000,0.20000,0.65000,14.51500,5.84500,1.28500,0.42900,0.03000,0.05000,0.06400,0.73000,1.07500,0.05600,0.56700,0.40000,1.50000,1.56000,19.35500,4.95000,1.73000,1.80000,4.80000,3.15000,10.66500,10.46500,16.64500,5.75000,10.37000
1,9.10000,13.53300,40.90000,8.77000,9.26500,6.01500,10.07000,7.15000,1.10700,1.02700,1.84800,8.62500,12.17500,0.28000,0.44000,0.08900,1.36000,0.06400,0.90200,1.45400,0.05600,0.19000,2.34700,5.10500,2.09500,19.35500,20.05000,9.53000,7.69500,4.55000,6.95000,2.04000,5.20000,5.72500,2.95000,2.23000
2,8.21000,3.71000,5.42000,8.77000,9.26500,4.55000,10.07000,7.15000,0.02000,0.02000,0.65000,17.14400,24.64500,0.38000,0.44000,0.06000,0.05000,0.08200,0.90200,2.02500,0.05600,0.56700,1.70000,1.70000,3.96000,4.90000,4.58000,3.02500,3.96000,4.93500,4.95000,4.72500,6.07500,6.75000,3.50000,3.17000
3,8.39000,8.70000,8.10000,9.50000,9.20000,5.20000,8.67000,6.67000,0.28000,0.27000,1.73000,3.87000,8.41000,1.48000,1.38000,0.05000,0.05000,0.07000,0.53000,1.74000,0.05000,0.06400,1.50000,1.50000,2.02000,3.96000,8.45000,2.07000,1.73000,6.30000,4.70000,3.50000,6.20000,8.67000,2.90000,7.37000
4,8.07000,8.05000,8.65000,7.96000,9.26500,3.29000,10.07000,7.15000,0.36000,0.43500,0.65000,3.85000,5.84500,0.28000,0.44000,0.10500,0.11500,0.07400,1.25200,1.07500,0.07100,0.19000,1.05000,1.15000,2.09500,3.90200,2.02000,1.73000,0.76000,4.80000,4.97000,3.95000,2.80000,8.40000,3.50000,3.90000


In [4]:
train.info(verbose=False)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3500 entries, 0 to 3499
Columns: 36 entries, Target to Bod5_7
dtypes: float16(36)
memory usage: 273.4 KB


<p style="
    font-size: 20px;
    font-family: 'JetBrains Mono';
    color: #4A4B52;
    border-bottom: 2px solid #FF7F51;
">
    <b>Features Description</b> 📔
</p>

<p style="
    font-size: 16px;
    font-family: 'JetBrains Mono';
">
    Features are divided into five major indicators of river water quality. These are:
</p>

<ul style="
    font-size: 16px;
    font-family: 'JetBrains Mono';
">
    <li><code>O2-Variables</code> - Dissolved oxygen (O2) is measured in mgO2/cub. dm (ie milligrams of oxygen (O2) in the cubic decimeter).</li>
    <li><code>Nh4-Variables</code> - Ammonium ions (NH4) concentration is measured in mg/cub. dm (ie milligrams in the cubic decimeter).</li>
    <li><code>No2-Variables</code> - Nitrite ions (NO2) concentration is measured in mg/cub. dm (ie milligrams in the cubic decimeter).</li>
    <li><code>No3-Variables</code> - Nitrate ions (NO3) concentration is measured in mg/cub. dm (ie milligrams in the cubic decimeter).</li>
    <li><code>Bod5-Variables</code> - Biochemical oxygen demand, which is determined in 5 days ("BOD5" or "BOD"). BOD5 is measured in mgO/cub. dm (ie milligrams of oxygen in the cubic decimeter).</li>
</ul>
<p style="
    font-size: 16px;
    font-family: 'JetBrains Mono';
">
    Each of the above groups is divided by several sub-features, i.e. $1-7$ - values of monthly averaged data in stations $1-7$ (in seven stations located from the target station upstream). Moreover, the <code>Target</code> is a value of monthly averaged data of O2 in the target station in mgO2/cub. dm.<br><br>
    <b>The dataset is composed of $3500$ samples and $35$ features, which gives us $100$ samples per dimension.</b>
</p>

In [5]:
assert len(train.columns) == 5 * 7 + 1  # Five major indicators from 7 stations and the target.


<p style="
    font-size: 20px;
    font-family: 'JetBrains Mono';
    color: #4A4B52;
    border-bottom: 2px solid #FF7F51;
">
    <b>Insight</b> 💡
</p>

<p style="
    font-size: 16px;
    font-family: 'JetBrains Mono';
">
    Reading the description of the original dataset, we know that the numbering of stations in the dataset is done from the target station upstream, i.e. closest to it - first, upstream - second, etc. <b>The assignment in the original dataset is to predict O2 values in the last station, in this case, station $0$. The water flows from station $2$ to $1$ and from $1$ to $0$. Therefore, O2 features should be good predictors for the task, shouldn't they?</b> We'll investigate this later. 
</p>

## <b> <span style="font-family: 'JetBrains Mono'; color: #4A4B52">1.2</span> <span style='color: #FF7F51'>|</span> <span style="font-family: 'JetBrains Mono'; color: #4A4B52">Basic Numerical Relations &amp; Summaries</span></b><a class="anchor" id="basic_numerical_relations_summaries"></a> [↑](#top)

<p style="
    font-size: 16px;
    font-family: 'JetBrains Mono';
">
    <b>Now we're going to look more closely at numerical properties and dependencies. Let's have a look at the target variable distribution first.</b>
</p>

In [26]:
target = train.Target
Q1, Q3 = np.percentile(target, 25), np.percentile(target, 75)

fig = px.histogram(
    x=target,
    labels={"x": "Target"},
    marginal="box",
    histnorm="probability density",
    title="Monthly Averaged Data of O\u2082 in Target Station<br>"
    "<span style='font-size: 75%; font-weight: bold;'>"
    "In the dataset, two significantly higher values have been observed. Outliers?</span>",
    color_discrete_sequence=["#4A4B52"],
    opacity=0.75,
    height=540,
    width=840,
)
fig.add_annotation(
    x=20,
    y=0.6,
    align="left",
    xanchor="left",
    text=f"<b>The major (75%) of the target values are in ({Q1:.2f}, {Q3:.2f}) range.</b>",
    showarrow=False,
)
fig.add_vrect(x0=Q1, x1=Q3, line_width=0, fillcolor="#FF7F51", opacity=0.25, row=1)
fig.add_vline(x=Q1, line_width=2, line_dash="dash", line_color="#4A4B52", row=1)
fig.add_vline(x=Q3, line_width=2, line_dash="dash", line_color="#4A4B52", row=1)
fig.update_yaxes(title="Probability Density", row=1)
fig.update_layout(
    font_color=FONT_COLOR,
    title_font_size=18,
    plot_bgcolor=BACKGROUND_COLOR,
    paper_bgcolor=BACKGROUND_COLOR,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    bargap=0.25,
)
fig.show()


<p style="
    font-size: 20px;
    font-family: 'JetBrains Mono';
    color: #4A4B52;
    border-bottom: 2px solid #FF7F51;
">
    <b>Target Distribution - Observations</b> 📜
</p>

<p style="
    font-size: 16px;
    font-family: 'JetBrains Mono';
">
    The <code>Target</code> feature has an extremely long tail, with at least two significant observations responsible for this state. <b>Perhaps we can reject these two samples, provided that we suppose the test dataset hasn't got observations like that. Otherwise, we should keep them.</b> The whole distribution is between $1.30$ and $65.93$ but $75$% of it is in the $(7.47, 9.11)$ range.<br><br>
    Let's get to the basic numerical summary of the dataset. Firstly, let's see the diversity of the dataset, i.e. missing values, unique values and their frequency.
</p>

In [29]:
missing_vals = train.isna().sum()

unique_vals = train.apply(lambda col: len(col.unique()))
most_freq_count = train.apply(lambda col: col.value_counts().iloc[0])
most_freq_val = train.mode().T.squeeze()

unique_ratio = unique_vals / len(train)
freq_count_ratio = most_freq_count / len(train)

basic_summary = pd.DataFrame(
    {
        "MissingValues": missing_vals,
        "UniqueValues": unique_vals,
        "UniqueValuesRatio": unique_ratio,
        "MostFreqValueCount": most_freq_count,
        "MostFreqValueCountRatio": freq_count_ratio,
        "MostFreqValue": most_freq_val,
    }
)

basic_summary.style.set_table_styles(DF_STYLE).background_gradient(DF_CMAP).format(
    {"UniqueValuesRatio": "{:.1%}", "MostFreqValueCountRatio": "{:.1%}"}, precision=3
)


,MissingValues,UniqueValues,UniqueValuesRatio,MostFreqValueCount,MostFreqValueCountRatio,MostFreqValue
Target,0,462,13.2%,106,3.0%,8.080
O2_1,0,329,9.4%,250,7.1%,7.500
O2_2,0,258,7.4%,321,9.2%,8.500
O2_3,0,235,6.7%,900,25.7%,9.545
O2_4,0,241,6.9%,677,19.3%,9.265
O2_5,0,317,9.1%,430,12.3%,6.015
O2_6,0,238,6.8%,844,24.1%,8.980
O2_7,0,303,8.7%,791,22.6%,7.150
Nh4_1,0,152,4.3%,483,13.8%,0.300
Nh4_2,0,157,4.5%,781,22.3%,0.200


<p style="
    font-size: 20px;
    font-family: 'JetBrains Mono';
    color: #4A4B52;
    border-bottom: 2px solid #FF7F51;
">
    <b>Unique &amp; Most Frequent Values - Observations</b> 📜
</p>

<p style="
    font-size: 16px;
    font-family: 'JetBrains Mono';
">
    As we know, in the dataset, there are $3500$ samples, but <b>the above table shows us how little we have unique values there. Probably, the synthetic data generation process is liable for that.</b> The most diverse feature is the <code>Target</code>, with about $13$% of unique values. On the other hand, <code>No2-Variables</code> are least varied in the dataset - usually $2$% of unique values. Also interesting are the most frequent values. There we have <code>Nh4_7</code> feature, where the only one value consist of above $50$% of that feature. <b>Maybe binarization will be fine for some of them?</b> Such weak diversity should be reflected in the distribution plots, which we see later.<br><br>
    Let's have a look at the numerical summary yet.
</p>

In [30]:
num_descr = (
    train.describe(percentiles=[0.01, 0.05, 0.25, 0.50, 0.75, 0.95, 0.99])
    .T.drop("count", axis=1)
    .rename(columns=str.title)
)

num_descr.style.set_table_styles(DF_STYLE).format(precision=3)


,Mean,Std,Min,1%,5%,25%,50%,75%,95%,99%,Max
Target,8.474,1.886,1.300,5.220,6.680,7.470,8.280,9.110,10.590,15.780,65.930
O2_1,8.217,3.041,0.000,4.200,5.300,7.100,7.890,9.100,10.700,14.901,46.950
O2_2,9.292,6.818,0.000,4.699,4.980,7.300,8.300,8.700,13.700,40.900,65.950
O2_3,9.633,1.439,4.900,6.239,7.960,8.770,9.500,9.545,12.070,14.900,16.900
O2_4,8.066,1.464,2.300,5.200,6.100,6.830,7.980,9.265,9.545,11.475,21.800
O2_5,5.672,2.721,0.200,0.636,1.910,4.550,5.800,7.015,9.431,12.030,59.400
O2_6,9.461,1.140,0.000,7.150,8.260,8.980,9.430,10.070,10.440,12.420,40.190
O2_7,6.547,1.663,0.000,2.310,4.158,5.877,6.430,7.150,8.980,11.002,15.900
Nh4_1,0.341,0.285,0.020,0.030,0.095,0.214,0.247,0.370,0.897,1.485,4.200
Nh4_2,0.398,0.452,0.020,0.020,0.090,0.200,0.245,0.405,1.200,2.600,3.600


<p style="
    font-size: 20px;
    font-family: 'JetBrains Mono';
    color: #4A4B52;
    border-bottom: 2px solid #FF7F51;
">
    <b>Numerical Summary - Observations</b> 📜
</p>

<p style="
    font-size: 16px;
    font-family: 'JetBrains Mono';
">
    At first sight, there we have a relevant outlier in the <code>Nh4_5</code> feature. <b>Even if the 99th percentile is about $30.26$, the maximum value is $3026$, which looks like a multiplication of it. Maybe it's something like an error?</b> As for the rest, there is rather nothing suspect. We will see that once more time and more closely on distribution plots later.<br><br>
    As for now let's have a look at the correlation matrix and hierarchical clustering. We should pay special attention to the first column (correlations with the target variable). 
</p>

In [31]:
pearson_corr = train.corr(method="pearson").round(2)
lower_triangular_corr = (
    pearson_corr.mask(np.triu(np.ones_like(pearson_corr, dtype=bool)))
    .dropna(axis="index", how="all")
    .dropna(axis="columns", how="all")
)

color_map = [[0.0, "#FCFCFC"], [0.5, "#4A4B52"], [1.0, "#FF7F51"]]

heatmap = go.Heatmap(
    z=lower_triangular_corr,
    x=lower_triangular_corr.columns,
    y=lower_triangular_corr.index,
    text=lower_triangular_corr.fillna(""),
    texttemplate="%{text}",
    xgap=2,
    ygap=2,
    showscale=True,
    colorscale=color_map,
    colorbar_len=1.02,
    hoverinfo="none",
)
fig = go.Figure(heatmap)
fig.add_annotation(
    x=9.7,
    y=5.3,
    align="left",
    xanchor="left",
    text="<b>Features seem to be rather uncorrelated. We can<br>"
    "distinct only several pairs with high correlation. Probably<br>"
    "majority of features are poor predictors for the target variable.</b>",
    showarrow=False,
)
fig.update_yaxes(tickfont_size=10)
fig.update_xaxes(tickfont_size=10)
fig.update_layout(
    font_color=FONT_COLOR,
    title="Lower Triangular of Correlation Matrix (Pearson)<br>"
    "<span style='font-size: 75%; font-weight: bold;'>"
    "Only O2_1 and O2_2 are somewhat correlated with the Target (O2_0)</span>",
    title_font_size=18,
    plot_bgcolor=BACKGROUND_COLOR,
    paper_bgcolor=BACKGROUND_COLOR,
    width=840,
    height=840,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    yaxis_autorange="reversed",
)
fig.show()


In [32]:
highest_abs_corr = (
    lower_triangular_corr.abs()
    .unstack()
    .sort_values(ascending=False)  # type: ignore
    .rename("Absolute Pearson Correlation")
    .to_frame()
    .reset_index(names=["Feature 1", "Feature 2"])
)

top_corr = highest_abs_corr[highest_abs_corr["Absolute Pearson Correlation"] > 0.5]
top_corr.style.set_table_styles(DF_STYLE).format(precision=2)


,Feature 1,Feature 2,Absolute Pearson Correlation
0,Nh4_1,Nh4_2,0.84
1,No3_1,No3_2,0.72
2,No3_6,No3_7,0.71
3,Bod5_1,Bod5_2,0.64
4,No3_3,No3_6,0.64
5,No3_4,No3_5,0.57
6,Bod5_6,Bod5_7,0.56
7,No3_3,No3_7,0.56
8,No3_3,No3_5,0.54


<p style="
    font-size: 20px;
    font-family: 'JetBrains Mono';
    color: #4A4B52;
    border-bottom: 2px solid #FF7F51;
">
    <b>Correlation Matrix - Observations</b> 📜
</p>

<p style="
    font-size: 16px;
    font-family: 'JetBrains Mono';
">
    As we can see, only <code>O2_1</code> and <code>O2_2</code> are somewhat correlated with the target variable ($0.48$ and $0.22$, respectively). When we think about it, this seems to be logical since these stations are located close to each other, and features represent the same variable as the target one. Also, <code>O2_2</code> is less correlated with the target (this station is further away).<b>I also checked the Spearman correlation, but the situation is similar to the Pearson one, i.e. weakly correlated features.</b> Moreover, most correlated pairs are related to the <code>No3-Variables</code>.<br><br>
    Since we have the correlation matrix, we can utilise it for another visualisation - hierarchical clustering. Nevertheless, relying on correlations requires an additional step because correlation measures the similarity between variables, but in hierarchical clustering, we need to provide dissimilarity (distance) between them. We can treat dissimilarity as $\textrm{dissimilarity} = 1 - \textrm{abs(correlation)}$. And basically, that's all. We can pass dissimilarity to the <code>linkage()</code> function from the <code>scipy</code> module and get clustering results. Just like as below.
</p>

In [11]:
dissimilarity = 1 - np.abs(pearson_corr)

fig = ff.create_dendrogram(
    dissimilarity,
    labels=pearson_corr.columns,
    orientation="left",
    colorscale=px.colors.sequential.Greys[1:],
    # squareform() returns lower triangular in compressed form - as 1D array.
    linkagefun=lambda x: linkage(squareform(dissimilarity), method="complete"),
)
fig.update_xaxes(showline=False, title="Distance", ticks="", range=[-0.05, 1.05], tickfont_size=10)
fig.update_yaxes(showline=False, ticks="", tickfont_size=10)
fig.update_layout(
    font_color=FONT_COLOR,
    title="Hierarchical Clustering using Correlation Matrix (Pearson)<br>"
    "<span style='font-size: 75%; font-weight: bold;'>"
    "Lack of closely related variables</span>",
    title_font_size=18,
    plot_bgcolor=BACKGROUND_COLOR,
    paper_bgcolor=BACKGROUND_COLOR,
    height=840,
    width=840,
)
fig.update_traces(line_width=1.25, opacity=1)
fig.show()


<p style="
    font-size: 20px;
    font-family: 'JetBrains Mono';
    color: #4A4B52;
    border-bottom: 2px solid #FF7F51;
">
    <b>Hierarchical Clustering - Observations</b> 📜
</p>

<p style="
    font-size: 16px;
    font-family: 'JetBrains Mono';
">
    Here, we should remember that we rely on the <b>Pearson</b> correlation. It measures linear dependency, and it's computed on actual values. However, we could have used, for example, the <b>Spearman</b> correlation, which is based on ranks and measures monotonic relations. Apart from that, we chose the <code>complete</code> method in the <code>linkage()</code> function, and if you take a different method, you can get different results.<br><br>
    <b>In this specific example, we see that features are rather in the long distance, which indicates weak relations. Nevertheless, we can spot that clusters are made between variables for nearby located stations (for example third and fourth, first and second, etc.).</b> It means that stations that are close to each other are probably good predictors for the variable from the nearby station. For example, <code>No3_1</code> is a good predictor for the <code>No3_2</code>, and so on.<br><br>
    Now, it's time to have a look at scatter plots for the highly correlated pairs. Let's get started with the <code>Target</code> and <code>O2_1</code> and <code>O2_2</code>.
</p>

In [12]:
fig = make_subplots(rows=1, cols=2, y_title="Target", shared_yaxes=True)
fig.update_annotations(font_size=14)

for col, component in enumerate(("O2_1", "O2_2"), start=1):
    fig.add_scatter(
        name=component,
        x=train[component],
        y=train.Target,
        mode="markers",
        marker_size=np.sqrt(train.Target + train[component]) * 0.8,
        marker=dict(color="#6B6A6A", symbol="circle", opacity=0.75),
        hovertemplate="Feature=%{x}<br>Target=%{y}",
        showlegend=False,
        row=1,
        col=col,
    )
    fig.add_scatter(
        name="Perfect Covering",
        x=[0, 70],
        y=[0, 70],
        mode="lines",
        marker=dict(color="#FF7F51"),
        showlegend=False,
        row=1,
        col=col,
    )
    fig.update_xaxes(showgrid=False, col=col, title=component, title_font_size=14, range=[0, 70])
    fig.update_yaxes(showgrid=False, col=col, range=[0, 70])

fig.update_layout(
    title="O\u2082 in Target Station vs Highest Correlated Variables<br>"
    "<span style='font-size: 75%; font-weight: bold;'>"
    "Only O2_1 (0.48) and O2_2 (0.22) are correlated with the Target (O2_0)</span>",
    font_color=FONT_COLOR,
    title_font_size=18,
    plot_bgcolor=BACKGROUND_COLOR,
    paper_bgcolor=BACKGROUND_COLOR,
    width=840,
    height=480,
)
fig.show()


<p style="
    font-size: 20px;
    font-family: 'JetBrains Mono';
    color: #4A4B52;
    border-bottom: 2px solid #FF7F51;
">
    <b>Target Pair Plots - Observations</b> 📜
</p>

<p style="
    font-size: 16px;
    font-family: 'JetBrains Mono';
">
    As we can see above, <code>O2_1</code> and <code>O2_2</code> seem to be good predictors for the target variable. <b>However, all of them contain some of outliers. Once more time, we can reject such samples as long as we suppose the test dataset is free from such observations.</b> Moreover, since the plot maintains an equal aspect ratio on axes, the orange line shows us when samples are perfectly covered. It shows that even if we can consider some sample outliers, they are still ideal predictors.
</p>

## <b> <span style="font-family: 'JetBrains Mono'; color: #4A4B52">1.3</span> <span style='color: #FF7F51'>|</span> <span style="font-family: 'JetBrains Mono'; color: #4A4B52">Distributions</span></b><a class="anchor" id="distributions"></a> [↑](#top)

<p style="
    font-size: 16px;
    font-family: 'JetBrains Mono';
">
    <b>In this subsection, we will see distributions of the available features. Each plot will be focused on only one indicator from all stations. Since we know that the dataset is weakly diversed and has few unique values, we should observe these distributions as a collection of individual peaks.</b>
</p>

In [13]:
def get_n_rows_axes(n_features, n_cols):
    n_rows = int(np.ceil(n_features / n_cols))
    current_col = range(1, n_cols + 1)
    current_row = range(1, n_rows + 1)
    return n_rows, list(product(current_row, current_col))


def draw_dist_for_group(group, vars_to_plot, n_cols=3, height=640):
    n_rows, axes = get_n_rows_axes(len(vars_to_plot), n_cols)
    fig = make_subplots(
        rows=n_rows,
        cols=n_cols,
        y_title="Probability Density",
        horizontal_spacing=0.1,
        vertical_spacing=0.1,
    )
    fig.update_annotations(font_size=14)
    for var, (row, col) in zip(vars_to_plot, axes):
        density, bins = np.histogram(train[var], bins=200, density=True)
        fig.add_bar(
            x=bins,
            y=density,
            marker_color="#6B6A6A",
            marker_line_width=0,
            opacity=0.5,
            name=var,
            showlegend=False,
            row=row,
            col=col,
        )
        fig.update_xaxes(
            tickfont_size=10,
            showgrid=False,
            title_text=var,
            titlefont_size=10,
            titlefont_family="Arial Black",
            row=row,
            col=col,
        )
        fig.update_yaxes(tickfont_size=10, showgrid=False, row=row, col=col)

    fig.update_layout(
        width=840,
        height=height,
        title=f"{group} Features - Probability Density",
        font_color=FONT_COLOR,
        title_font_size=18,
        plot_bgcolor=BACKGROUND_COLOR,
        paper_bgcolor=BACKGROUND_COLOR,
        bargap=0,
    )
    return fig


o2_group = "O2"
o2_vars = train.columns[train.columns.str.startswith(o2_group)]

fig = draw_dist_for_group(o2_group, o2_vars)
fig.show()


In [14]:
nh4_group = "Nh4"
nh4_vars = train.columns[train.columns.str.startswith(nh4_group)]

fig = draw_dist_for_group(nh4_group, nh4_vars)
fig.show()

In [15]:
no2_group = "No2"
no2_vars = train.columns[train.columns.str.startswith(no2_group)]

fig = draw_dist_for_group(no2_group, no2_vars)
fig.show()

In [16]:
no3_group = "No3"
no3_vars = train.columns[train.columns.str.startswith(no3_group)]

fig = draw_dist_for_group(no3_group, no3_vars)
fig.show()

In [17]:
bod5_group = "Bod5"
bod5_vars = train.columns[train.columns.str.startswith(bod5_group)]

fig = draw_dist_for_group(bod5_group, bod5_vars)
fig.show()

<p style="
    font-size: 20px;
    font-family: 'JetBrains Mono';
    color: #4A4B52;
    border-bottom: 2px solid #FF7F51;
">
    <b>Features Distributions - Observations</b> 📜
</p>

<p style="
    font-size: 16px;
    font-family: 'JetBrains Mono';
">
    Unfortunately, feature distributions are not so helpful in this specific problem. Above, we see the confirmation of what we've talked about earlier, i.e. lack of diversity in the dataset. Most of the data is built from samples with fixed values.<br><br>
    <b>Usually, when it comes to continuous features, we can investigate so-called probability plots. These visualizations show us whether a certain feature follows the normal one.</b> We find out about that through the coefficient of determination and equally deployed samples around a straight line placed on this plot. Sometimes fixing the fit to the normal distribution gives an excellent boost to the machine learning model performance. Such a situation appears, for example, for the SVM and Logistic Regression. <b>We can fix normality through some simple transformations like log-level ones for distributions with long tails or square ones for left-skewed data. Also, there are more sophisticated methods like Box-Cox transformation or Yeo-Johnson one.</b> Nevertheless, as for now, we won't be doing that in this notebook since the tree-based model probably won't benefit so much from it.
</p>

## <b> <span style="font-family: 'JetBrains Mono'; color: #4A4B52">1.4</span> <span style='color: #FF7F51'>|</span> <span style="font-family: 'JetBrains Mono'; color: #4A4B52">Quick Summary</span></b><a class="anchor" id="quick_summary"></a> [↑](#top)

<p style="
    font-size: 20px;
    font-family: 'JetBrains Mono';
    color: #4A4B52;
    border-bottom: 2px solid #FF7F51;
">
    <b>What We Already Know</b> 📔
</p>

<p style="
    font-size: 16px;
    font-family: 'JetBrains Mono';
">
    <b>The first section provides a quick overview of the available dataset. In this section we found out several things:</b>
</p>

<ul style="
    font-size: 16px;
    font-family: 'JetBrains Mono';
">
    <li>This is a data cleaning (data generation) competition with a fixed model (random forest) evaluated on the hidden test dataset. We need to provide a training dataset to train the model, which next is evaluated using the <b>RMSE</b> metric.</li>
    <li><b>The general problem is focused on dissolved oxygen prediction in river water.</b> Measurements are stored from seven different stations located on the same river, and the original task is to predict O2 values in the last station.</li>
    <li><b>The dataset we're working on was generated synthetically using the original one.</b> There, we have $3500$ samples and $35$ features (seven stations and five indicators from each of them, which gives us $35$ features indeed). <b>This gives us $100$ samples per dimension.</b> In such a case, there is rather no curse of dimensionality, but still some of features probably can introduce a noise to the machine learning models.</li>
    <li>The target, i.e. monthly averaged data of O2 in the target station, has a long tail due to two significantly larger observations. The whole distribution is between $1.30$ and $65.93$, but $75$% of it is in the $(7.47, 9.11)$ range. Probably, this gives us hope that the target variable can be clipped.</li>
    <li>Since the dataset was generated synthetically, there is a relative lack of diversity in the data. <b>Most of the data are fixed values, and we have only a small percentage of unique observations.</b> Nevertheless, there are no missing values, so we don't need to bother about the appropriate imputation process.</li>
    <li>There is one feature (apart from the target variable), i.e. <code>Nh4_5</code>, where a relevant outlier has been recorded. The maximum value is $3026$ but the $99$th percentile is $30.26$, so it looks like an error, or something like a multiplication.</li>
    <li><b>Features are weakly correlated (it concerns both Pearson and Spearman methods).</b> For us, the most interesting is the target variable. This correlates with the <code>O2_1</code> ($0.48$) and <code>O2_2</code> ($0.22$). The rest of the features have practically zero correlation with the target. As for the remaining variables, the most correlated ones are <code>No3-Variables</code>. Weak relations (long distances) are also shown on the dendrogram visualization.</li>
    <li>Feature distributions are characterized by a collection of individual peaks. It's related to the large number of duplicated observations in the variable.</li>
</ul>

<p style="
    font-size: 16px;
    font-family: 'JetBrains Mono';
">
    Well, we already know more or less about the dataset. It seems that the most important features will be <code>O2_1</code> and <code>O2_2</code>. <b>The subsequent section answers this question.</b>
</p>

# <b> <span style="font-family: 'JetBrains Mono'; color: #4A4B52">2</span> <span style='color: #FF7F51'>|</span> <span style="font-family: 'JetBrains Mono'; color: #4A4B52">Feature Importances</span></b><a class="anchor" id="feature_importances"></a> [↑](#top)

<p style="
    font-size: 16px;
    font-family: 'JetBrains Mono';
">
    <b>This notebook will be updated regularly. I want to depict feature importance problems with feature permutation tests, adding random variables, utilising the shap library, reducing dimensionality, showing partial dependence plots and so on. Subsequently investigate outliers with LOF and Isolation Forest.</b><br><br>
    I hope you like it, and if yes, then leave an upvote. If you have any questions, let me know in the comments section. Moreover, I encourage you to check my other notebooks, especially these: <a href="https://www.kaggle.com/code/mateuszk013/neural-machine-translation-with-attention"><b>Neural Machine Translation with Attention</b></a> and <a href="https://www.kaggle.com/code/mateuszk013/ml-explainability-shapley-values-shap"><b>ML Explainability - Shapley Values & SHAP</b></a>. These are important to me. Thanks and good luck!
</p>